In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean

In [2]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/25 18:07:56 WARN Utils: Your hostname, MacBook-Air-de-Esteban-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.133 instead (on interface en0)
24/04/25 18:07:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/25 18:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
districtes = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:../exploited_zone/freshdata_exploited2.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", '''
        SELECT *
        FROM nova_base
    ''') \
    .load()


In [9]:
districtes.show()

+----+---------+-------------+--------------------+----------------------------------+------------------+
| Any|Trimestre|Nom_Districte|           Nom_Barri|Superfície_mitjana_(m2_construïts)|            Nombre|
+----+---------+-------------+--------------------+----------------------------------+------------------+
|2023|        1|       Gràcia|   la_Vila_de_Gràcia|                             Total|              80.5|
|2023|        2|       Gràcia|   la_Vila_de_Gràcia|                             Total| 81.30000305175781|
|2023|        3|       Gràcia|   la_Vila_de_Gràcia|                             Total|  83.0999984741211|
|2023|        1|       Gràcia|el_Camp_d_en_Gras...|                             Total| 77.30000305175781|
|2023|        2|       Gràcia|el_Camp_d_en_Gras...|                             Total|  71.4000015258789|
|2023|        3|       Gràcia|el_Camp_d_en_Gras...|                             Total|              68.5|
|2023|        1|       Gràcia|             el_